# Traffic Forecasting with Deep Learning - HCMC

**Complete Pipeline**: Data Loading -> Preprocessing -> Feature Engineering -> DL Model Training

**Author:** thatlq1812  
**Project:** DSP391m Traffic Forecasting System  
**Models:** LSTM and ATSCGN (Deep Learning)

---

## Pipeline Overview

| Step | Description | Required |
|------|-------------|----------|
| 1 | Configuration - Set pipeline options | Yes |
| 2 | Data Loading - Load collected traffic data | Yes |
| 3 | Data Exploration - Preview and understand data | Optional |
| 4 | Preprocessing - Clean and prepare data | Yes |
| 5 | Feature Engineering - Create features for DL models | Yes |
| 6 | Model Training - Train LSTM and ATSCGN | Yes |
| 7 | Evaluation - Compare model performance | Yes |
| 8 | Save Models - Export for production | Optional |

---

**Coverage:** 78 nodes, 234 road segments, 4096m radius  
**Data Source:** Google Directions API + Open-Meteo Weather  
**VM:** traffic-forecast-collector (GCP asia-southeast1-a)

---
## Step 1: Pipeline Configuration

Configure which steps to run and analysis options.

In [1]:
# Pipeline Configuration
# ======================

# Data Source
DATA_DIR = '../data/runs'
PROCESSED_DIR = '../data/processed'
MODELS_DIR = '../models'

# Pipeline Steps Control
ENABLE_DATA_EXPLORATION = True      # Show data preview and statistics
ENABLE_PREPROCESSING = True         # Clean and prepare data
ENABLE_FEATURE_ENGINEERING = True   # Create features for models
ENABLE_MODEL_TRAINING = True        # Train LSTM and ATSCGN
ENABLE_EVALUATION = True            # Evaluate and compare models
ENABLE_SAVE_MODELS = True           # Save trained models

# Model Configuration
TRAIN_LSTM = True                   # Train LSTM model
TRAIN_ATSCGN = True                 # Train ATSCGN model

# Training Parameters
EPOCHS = 50                         # Number of training epochs
BATCH_SIZE = 32                     # Batch size for training
LEARNING_RATE = 0.001               # Learning rate
VALIDATION_SPLIT = 0.2              # Validation data percentage
SEQUENCE_LENGTH = 12                # Input sequence length (12 = 3 hours at 15-min intervals)

# Analysis Options
VERBOSE = True                      # Print detailed progress
PLOT_TRAINING_HISTORY = True        # Plot training curves

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Configuration loaded successfully!")
print("\nPipeline Configuration:")
print("=" * 60)
print(f"Data Exploration:        {'Enabled' if ENABLE_DATA_EXPLORATION else 'Disabled'}")
print(f"Preprocessing:           {'Enabled' if ENABLE_PREPROCESSING else 'Disabled'}")
print(f"Feature Engineering:     {'Enabled' if ENABLE_FEATURE_ENGINEERING else 'Disabled'}")
print(f"Model Training:          {'Enabled' if ENABLE_MODEL_TRAINING else 'Disabled'}")
print(f"  - LSTM:                {'Yes' if TRAIN_LSTM else 'No'}")
print(f"  - ATSCGN:              {'Yes' if TRAIN_ATSCGN else 'No'}")
print(f"Evaluation:              {'Enabled' if ENABLE_EVALUATION else 'Disabled'}")
print(f"Save Models:             {'Enabled' if ENABLE_SAVE_MODELS else 'Disabled'}")
print("\nTraining Parameters:")
print("=" * 60)
print(f"Epochs:                  {EPOCHS}")
print(f"Batch Size:              {BATCH_SIZE}")
print(f"Learning Rate:           {LEARNING_RATE}")
print(f"Validation Split:        {VALIDATION_SPLIT}")
print(f"Sequence Length:         {SEQUENCE_LENGTH}")
print("=" * 60)

Configuration loaded successfully!

Pipeline Configuration:
Data Exploration:        Enabled
Preprocessing:           Enabled
Feature Engineering:     Enabled
Model Training:          Enabled
  - LSTM:                Yes
  - ATSCGN:              Yes
Evaluation:              Enabled
Save Models:             Enabled

Training Parameters:
Epochs:                  50
Batch Size:              32
Learning Rate:           0.001
Validation Split:        0.2
Sequence Length:         12


---
## Step 2: Import Required Libraries

Import all necessary libraries for data processing and modeling.

In [2]:
# Core Libraries
import sys
import os
from pathlib import Path
import json
from datetime import datetime, timedelta

# Data Processing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Deep Learning
try:
    import tensorflow as tf
    from tensorflow import keras
    print(f"TensorFlow version: {tf.__version__}")
    HAS_TENSORFLOW = True
except ImportError:
    print("TensorFlow not installed. Please install: pip install tensorflow")
    HAS_TENSORFLOW = False

# Project Modules
sys.path.append('..')
from traffic_forecast.ml import DataLoader, DataPreprocessor
from traffic_forecast.models import registry

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("\nLibraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

TensorFlow version: 2.18.1

Libraries imported successfully!
NumPy version: 1.26.4
Pandas version: 2.1.4


---
## Step 3: Load Data

Load collected traffic data from JSON files.

In [3]:
# Load data using DataLoader
loader = DataLoader(data_dir=DATA_DIR)

print("Loading traffic data...")
print("=" * 60)

# Get available runs
available_runs = loader.list_available_runs()
print(f"Found {len(available_runs)} collection runs")

if len(available_runs) == 0:
    print("\nNo data found! Please collect data first:")
    print("  python scripts/collect_once.py")
    raise ValueError("No data available")

# Load all available data
data = loader.load_all_runs()

print(f"\nData loaded successfully!")
print(f"Total records: {len(data)}")
print(f"Date range: {data['timestamp'].min()} to {data['timestamp'].max()}")
print(f"\nColumns: {list(data.columns)}")
print("\nFirst few records:")
print(data.head())

AttributeError: 'str' object has no attribute 'exists'

---
## Step 4: Data Exploration (Optional)

Explore and understand the loaded data.

In [ ]:
if ENABLE_DATA_EXPLORATION:
    print("Data Exploration")
    print("=" * 60)
    
    # Basic statistics
    print("\n1. Dataset Info:")
    print(f"   Shape: {data.shape}")
    print(f"   Memory usage: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\n2. Data Types:")
    print(data.dtypes)
    
    print("\n3. Missing Values:")
    missing = data.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("   No missing values")
    
    print("\n4. Statistical Summary:")
    print(data.describe())
    
    # Visualizations
    if 'speed_kmh' in data.columns:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Speed distribution
        axes[0, 0].hist(data['speed_kmh'], bins=50, edgecolor='black')
        axes[0, 0].set_title('Speed Distribution')
        axes[0, 0].set_xlabel('Speed (km/h)')
        axes[0, 0].set_ylabel('Frequency')
        
        # Speed over time
        data_sample = data.sample(min(1000, len(data))).sort_values('timestamp')
        axes[0, 1].scatter(data_sample['timestamp'], data_sample['speed_kmh'], alpha=0.5)
        axes[0, 1].set_title('Speed Over Time (Sample)')
        axes[0, 1].set_xlabel('Timestamp')
        axes[0, 1].set_ylabel('Speed (km/h)')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # Speed by hour (if hour column exists)
        if 'hour' in data.columns:
            hourly_speed = data.groupby('hour')['speed_kmh'].mean()
            axes[1, 0].plot(hourly_speed.index, hourly_speed.values, marker='o')
            axes[1, 0].set_title('Average Speed by Hour')
            axes[1, 0].set_xlabel('Hour of Day')
            axes[1, 0].set_ylabel('Average Speed (km/h)')
            axes[1, 0].grid(True)
        
        # Speed boxplot by day of week (if available)
        if 'day_of_week' in data.columns:
            data.boxplot(column='speed_kmh', by='day_of_week', ax=axes[1, 1])
            axes[1, 1].set_title('Speed Distribution by Day of Week')
            axes[1, 1].set_xlabel('Day of Week')
            axes[1, 1].set_ylabel('Speed (km/h)')
        
        plt.tight_layout()
        plt.show()
    
    print("\nData exploration complete!")
else:
    print("Data exploration skipped (ENABLE_DATA_EXPLORATION = False)")

---
## Step 5: Data Preprocessing

Clean and prepare data for model training.

In [ ]:
if ENABLE_PREPROCESSING:
    print("Data Preprocessing")
    print("=" * 60)
    
    # Initialize preprocessor
    preprocessor = DataPreprocessor()
    
    # Preprocess data
    print("\nCleaning and transforming data...")
    data_processed = preprocessor.fit_transform(data)
    
    print(f"\nPreprocessing complete!")
    print(f"Records before: {len(data)}")
    print(f"Records after: {len(data_processed)}")
    print(f"Columns: {list(data_processed.columns)}")
    
    # Save preprocessed data
    os.makedirs(PROCESSED_DIR, exist_ok=True)
    output_file = os.path.join(PROCESSED_DIR, 'traffic_data_processed.parquet')
    data_processed.to_parquet(output_file, index=False)
    print(f"\nSaved preprocessed data to: {output_file}")
    
    # Use processed data going forward
    data = data_processed
else:
    print("Preprocessing skipped (ENABLE_PREPROCESSING = False)")

---
## Step 6: Feature Engineering

Create features for deep learning models.

In [ ]:
if ENABLE_FEATURE_ENGINEERING:
    print("Feature Engineering for Deep Learning")
    print("=" * 60)
    
    from traffic_forecast.ml.features import (
        add_temporal_features,
        add_spatial_features,
        add_weather_features,
        add_traffic_features
    )
    
    # Add temporal features
    print("\n1. Adding temporal features...")
    data = add_temporal_features(data)
    print(f"   Added: hour, day_of_week, is_weekend, is_peak_hour")
    
    # Add spatial features (if node coordinates available)
    print("\n2. Adding spatial features...")
    try:
        data = add_spatial_features(data)
        print(f"   Added: distance, bearing, etc.")
    except Exception as e:
        print(f"   Skipped: {e}")
    
    # Add weather features (if available)
    print("\n3. Adding weather features...")
    try:
        data = add_weather_features(data)
        print(f"   Added: temperature, precipitation, etc.")
    except Exception as e:
        print(f"   Skipped: {e}")
    
    # Add traffic features (lag features)
    print("\n4. Adding traffic lag features...")
    try:
        data = add_traffic_features(data)
        print(f"   Added: speed lags, rolling averages")
    except Exception as e:
        print(f"   Skipped: {e}")
    
    print(f"\nFeature engineering complete!")
    print(f"Total features: {len(data.columns)}")
    print(f"Feature names: {list(data.columns)}")
    
    # Remove any remaining NaN values
    data = data.dropna()
    print(f"\nRecords after dropping NaN: {len(data)}")
else:
    print("Feature engineering skipped (ENABLE_FEATURE_ENGINEERING = False)")

---
## Step 7: Prepare Training Data

Create sequences for time-series prediction.

In [ ]:
def create_sequences(data, sequence_length, target_col='speed_kmh'):
    """Create sequences for time-series prediction."""
    X, y = [], []
    
    # Sort by timestamp
    data_sorted = data.sort_values('timestamp').reset_index(drop=True)
    
    # Select feature columns (exclude timestamp and target)
    feature_cols = [col for col in data_sorted.columns 
                    if col not in ['timestamp', target_col]]
    
    # Create sequences
    for i in range(len(data_sorted) - sequence_length):
        X.append(data_sorted[feature_cols].iloc[i:i+sequence_length].values)
        y.append(data_sorted[target_col].iloc[i+sequence_length])
    
    return np.array(X), np.array(y), feature_cols

print("Preparing training sequences...")
print("=" * 60)

# Create sequences
X, y, feature_names = create_sequences(data, SEQUENCE_LENGTH)

print(f"\nSequences created!")
print(f"X shape: {X.shape} (samples, timesteps, features)")
print(f"y shape: {y.shape}")
print(f"Number of features: {len(feature_names)}")

# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)

print(f"\nTrain-Test Split:")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

---
## Step 8: Train LSTM Model

Train LSTM model for traffic speed prediction.

In [ ]:
if ENABLE_MODEL_TRAINING and TRAIN_LSTM and HAS_TENSORFLOW:
    print("Training LSTM Model")
    print("=" * 60)
    
    # Build LSTM model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    
    model_lstm = Sequential([
        LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LENGTH, X_train.shape[2])),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
    model_lstm.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='mse',
        metrics=['mae']
    )
    
    print("\nModel Architecture:")
    model_lstm.summary()
    
    # Callbacks
    os.makedirs(os.path.join(MODELS_DIR, 'checkpoints'), exist_ok=True)
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ModelCheckpoint(
            os.path.join(MODELS_DIR, 'checkpoints', 'lstm_best.h5'),
            monitor='val_loss',
            save_best_only=True
        )
    ]
    
    # Train model
    print("\nTraining LSTM model...")
    history_lstm = model_lstm.fit(
        X_train, y_train,
        validation_split=VALIDATION_SPLIT,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1 if VERBOSE else 0
    )
    
    # Evaluate
    print("\nEvaluating LSTM model...")
    test_loss, test_mae = model_lstm.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss (MSE): {test_loss:.4f}")
    print(f"Test MAE: {test_mae:.4f} km/h")
    
    # Plot training history
    if PLOT_TRAINING_HISTORY:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Loss
        axes[0].plot(history_lstm.history['loss'], label='Training Loss')
        axes[0].plot(history_lstm.history['val_loss'], label='Validation Loss')
        axes[0].set_title('LSTM Model Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss (MSE)')
        axes[0].legend()
        axes[0].grid(True)
        
        # MAE
        axes[1].plot(history_lstm.history['mae'], label='Training MAE')
        axes[1].plot(history_lstm.history['val_mae'], label='Validation MAE')
        axes[1].set_title('LSTM Model MAE')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('MAE (km/h)')
        axes[1].legend()
        axes[1].grid(True)
        
        plt.tight_layout()
        plt.show()
    
    print("\nLSTM training complete!")
else:
    if not HAS_TENSORFLOW:
        print("LSTM training skipped (TensorFlow not available)")
    else:
        print("LSTM training skipped (TRAIN_LSTM = False)")

---
## Step 9: Train ATSCGN Model

Train ATSCGN (Adaptive Traffic Spatial-Temporal Convolutional Graph Network) model.

In [ ]:
if ENABLE_MODEL_TRAINING and TRAIN_ATSCGN and HAS_TENSORFLOW:
    print("Training ATSCGN Model")
    print("=" * 60)
    print("\nNote: ATSCGN requires graph adjacency matrix.")
    print("This is a placeholder for future implementation.")
    print("\nFor now, use LSTM for traffic forecasting.")
    print("ATSCGN implementation coming soon!")
    
    # TODO: Implement ATSCGN training
    # from traffic_forecast.models.graph import ASTGCNTrafficModel
    # model_atscgn = ASTGCNTrafficModel(...)
    
else:
    if not HAS_TENSORFLOW:
        print("ATSCGN training skipped (TensorFlow not available)")
    else:
        print("ATSCGN training skipped (TRAIN_ATSCGN = False)")

---
## Step 10: Model Evaluation and Comparison

Evaluate and compare model performance.

In [ ]:
if ENABLE_EVALUATION and TRAIN_LSTM and HAS_TENSORFLOW:
    print("Model Evaluation")
    print("=" * 60)
    
    # Make predictions
    y_pred_lstm = model_lstm.predict(X_test).flatten()
    
    # Calculate metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    mse = mean_squared_error(y_test, y_pred_lstm)
    mae = mean_absolute_error(y_test, y_pred_lstm)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_lstm)
    
    print("\nLSTM Model Performance:")
    print(f"  MSE:  {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} km/h")
    print(f"  MAE:  {mae:.4f} km/h")
    print(f"  R2:   {r2:.4f}")
    
    # Visualize predictions
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Predictions vs Actual (sample)
    sample_size = min(200, len(y_test))
    axes[0].plot(y_test[:sample_size], label='Actual', alpha=0.7)
    axes[0].plot(y_pred_lstm[:sample_size], label='Predicted', alpha=0.7)
    axes[0].set_title('LSTM Predictions vs Actual (Sample)')
    axes[0].set_xlabel('Sample Index')
    axes[0].set_ylabel('Speed (km/h)')
    axes[0].legend()
    axes[0].grid(True)
    
    # Scatter plot
    axes[1].scatter(y_test, y_pred_lstm, alpha=0.5)
    axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                 'r--', label='Perfect Prediction')
    axes[1].set_title('LSTM Predictions vs Actual (Scatter)')
    axes[1].set_xlabel('Actual Speed (km/h)')
    axes[1].set_ylabel('Predicted Speed (km/h)')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Error distribution
    errors = y_test - y_pred_lstm
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=50, edgecolor='black')
    plt.axvline(x=0, color='r', linestyle='--', label='Zero Error')
    plt.title('LSTM Prediction Error Distribution')
    plt.xlabel('Error (km/h)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("\nEvaluation complete!")
else:
    print("Evaluation skipped (ENABLE_EVALUATION = False or no trained model)")

---
## Step 11: Save Models

Save trained models for production use.

In [ ]:
if ENABLE_SAVE_MODELS and TRAIN_LSTM and HAS_TENSORFLOW:
    print("Saving Models")
    print("=" * 60)
    
    # Create models directory
    os.makedirs(os.path.join(MODELS_DIR, 'production'), exist_ok=True)
    
    # Save LSTM model
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    lstm_path = os.path.join(MODELS_DIR, 'production', f'lstm_model_{timestamp}.h5')
    model_lstm.save(lstm_path)
    print(f"\nLSTM model saved to: {lstm_path}")
    
    # Save model metadata
    metadata = {
        'model_type': 'LSTM',
        'timestamp': timestamp,
        'sequence_length': SEQUENCE_LENGTH,
        'num_features': X_train.shape[2],
        'feature_names': feature_names,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'test_mse': float(mse),
        'test_mae': float(mae),
        'test_rmse': float(rmse),
        'test_r2': float(r2)
    }
    
    metadata_path = os.path.join(MODELS_DIR, 'production', f'lstm_metadata_{timestamp}.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Model metadata saved to: {metadata_path}")
    
    print("\nModels saved successfully!")
else:
    print("Model saving skipped (ENABLE_SAVE_MODELS = False or no trained model)")

---
## Summary

Pipeline execution complete!

In [ ]:
print("\n" + "=" * 60)
print("PIPELINE EXECUTION SUMMARY")
print("=" * 60)
print(f"\nData:")
print(f"  Total records: {len(data)}")
print(f"  Features: {len(feature_names)}")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")

if TRAIN_LSTM and HAS_TENSORFLOW:
    print(f"\nLSTM Model:")
    print(f"  Test RMSE: {rmse:.4f} km/h")
    print(f"  Test MAE: {mae:.4f} km/h")
    print(f"  Test R2: {r2:.4f}")
    if ENABLE_SAVE_MODELS:
        print(f"  Model saved: {lstm_path}")

print("\n" + "=" * 60)
print("Next Steps:")
print("  1. Use saved model for predictions")
print("  2. Deploy to production (GCP VM)")
print("  3. Monitor model performance")
print("  4. Retrain with more data as needed")
print("=" * 60)

# Complete Traffic Forecasting Pipeline - HCMC

**All-in-One Workflow**: Download → Preprocess → EDA → Feature Engineering → Model Training
**Author:** thatlq1812

---

## Pipeline Overview

| Step | Description | Optional |
|------|-------------|----------|
| 1️ | **Configuration** - Set pipeline options | NOT Required |
| 2️ | **Download Data** - Get latest from VM | OK Skippable |
| 3️ | **Explore Data** - Preview raw data | OK Skippable |
| 4️ | **Preprocess** - Convert & add features | OK Skippable |
| 5️ | **Comprehensive EDA** - Deep analysis | OK Skippable |
| 6️ | **Feature Engineering** - ML features | NOT Required |
| 7️ | **Model Training** - Train & compare | NOT Required |
| 8️ | **Save Results** - Export models | OK Skippable |

---

**Project:** DSP391m Traffic Forecasting - Ho Chi Minh City  
**VM:** traffic-forecast-collector (GCP asia-southeast1-a)  
**Coverage:** 64 intersections, 144 road segments, 4096m radius

---

### Quick Start Tips:

- **First time?** Run all cells with default config
- **Have data already?** Set `USE_EXISTING_DATA = True`
- **Quick test?** Disable EDA steps, enable `QUICK_MODE = True`
- **Production?** Enable all steps for comprehensive analysis

---
## Step 1️: Pipeline Configuration

**Configure which steps to run** - Customize the pipeline to your needs.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# PIPELINE CONFIGURATION
# ═══════════════════════════════════════════════════════════════════

# ─── Data Source Options ───────────────────────────────────────────
USE_EXISTING_DATA = True       # True: Use current data | False: Download new data
DOWNLOAD_LATEST_ONLY = False   # True: Latest run only | False: All available runs
USE_PREPROCESSED = False       # True: Load from Parquet | False: Process from JSON

# ─── Pipeline Steps Control ────────────────────────────────────────
ENABLE_DATA_EXPLORATION = True    # Show raw data preview
ENABLE_PREPROCESSING = True       # Convert JSON to Parquet
ENABLE_COMPREHENSIVE_EDA = True   # Full exploratory analysis (maps, charts)
ENABLE_FEATURE_ENGINEERING = True # Create ML features (always needed for training)
ENABLE_MODEL_TRAINING = True      # Train and compare models
ENABLE_SAVE_MODELS = True         # Save trained models to disk

# ─── Analysis Options ──────────────────────────────────────────────
QUICK_MODE = False             # True: Faster execution, less detail
SHOW_INTERACTIVE_MAPS = True   # Folium geographic visualizations
SHOW_PLOTLY_CHARTS = True      # Interactive Plotly charts
VERBOSE = True                 # Print detailed progress

# ─── Data Paths ────────────────────────────────────────────────────
DATA_DIR = '../data/runs'
PROCESSED_DIR = '../data/processed'
MODELS_DIR = '../traffic_forecast/models/saved'

# ═══════════════════════════════════════════════════════════════════

import warnings
warnings.filterwarnings('ignore')

print("Configuration loaded!\n")
print("=" * 70)
print("Pipeline Configuration:")
print("=" * 70)
print(f"Data Source:")
print(f"   {'OK' if USE_EXISTING_DATA else 'NOT'} Use existing data (skip download)")
print(f"   {'OK' if DOWNLOAD_LATEST_ONLY else 'NOT'} Download latest only")
print(f"   {'OK' if USE_PREPROCESSED else 'NOT'} Use preprocessed Parquet files")
print(f"\nPipeline Steps:")
print(f"   {'OK' if ENABLE_DATA_EXPLORATION else 'NOT'} Data exploration")
print(f"   {'OK' if ENABLE_PREPROCESSING else 'NOT'} Preprocessing")
print(f"   {'OK' if ENABLE_COMPREHENSIVE_EDA else 'NOT'} Comprehensive EDA")
print(f"   {'OK' if ENABLE_FEATURE_ENGINEERING else 'NOT'} Feature engineering")
print(f"   {'OK' if ENABLE_MODEL_TRAINING else 'NOT'} Model training")
print(f"   {'OK' if ENABLE_SAVE_MODELS else 'NOT'} Save models")
print(f"\nAnalysis Options:")
print(f"   {'OK' if QUICK_MODE else 'NOT'} Quick mode (faster)")
print(f"   {'OK' if SHOW_INTERACTIVE_MAPS else 'NOT'} Interactive maps")
print(f"   {'OK' if SHOW_PLOTLY_CHARTS else 'NOT'} Plotly charts")
print(f"   {'OK' if VERBOSE else 'NOT'} Verbose output")
print("=" * 70)

# Validate configuration
if ENABLE_MODEL_TRAINING and not ENABLE_FEATURE_ENGINEERING:
    print("\nWARNING: Model training requires feature engineering!")
    print("   Automatically enabling ENABLE_FEATURE_ENGINEERING")
    ENABLE_FEATURE_ENGINEERING = True

if USE_PREPROCESSED and not ENABLE_PREPROCESSING:
    print("\nTIP: Using preprocessed data, skipping preprocessing step")

print("\nReady to run! Execute cells below to start the pipeline.")
print("Expected: 10 runs, 1,440 records, ~6-8 minutes execution time")

---
## Step 2️: Data Source Selection

Choose your data source based on the configuration above.

### Option A: Download Latest Data from VM

**Run this cell only if** `USE_EXISTING_DATA = False`

In [ ]:
if not USE_EXISTING_DATA:
    import subprocess
    import os
    from datetime import datetime
    
    print("Downloading latest data from VM...\n")
    print("=" * 70)
    
    # Run download script
    result = subprocess.run(
        ['bash', 'scripts/data/download_latest.sh'],
        capture_output=True,
        text=True,
        cwd='..'  # Run from project root
    )
    
    print(result.stdout)
    if result.returncode == 0:
        print("\nOK Download completed successfully!")
    else:
        print("\nFAIL Download failed!")
        print(result.stderr)
else:
    print("Skipping download - Using existing data")
    print("Set USE_EXISTING_DATA = False to download new data")

### Option B: Use Existing Data

Current data will be loaded from `data/runs/`

---
## Step 3️: Data Exploration

**Preview and validate** the data we'll be working with.

In [ ]:
if ENABLE_DATA_EXPLORATION:
    import os
    import json
    import pandas as pd
    from pathlib import Path
    from datetime import datetime
    
    # Find all runs
    data_dir = Path(DATA_DIR)
    run_dirs = sorted([d for d in data_dir.iterdir() if d.is_dir()], reverse=True)
    
    print(f"Found {len(run_dirs)} collection runs\n")
    print("=" * 80)
    
    # Display runs table
    runs_info = []
    for run_dir in run_dirs[:10]:  # Show latest 10
        files = list(run_dir.glob('*.json'))
        total_size = sum(f.stat().st_size for f in files)
        
        # Parse timestamp from run name
        run_name = run_dir.name
        if run_name.startswith('run_'):
            timestamp_str = run_name[4:]  # Remove 'run_' prefix
            try:
                run_time = datetime.strptime(timestamp_str, '%Y%m%d_%H%M%S')
                time_display = run_time.strftime('%Y-%m-%d %H:%M:%S')
            except:
                time_display = timestamp_str
        else:
            time_display = run_name
        
        runs_info.append({
            'Run': run_name,
            'Time': time_display,
            'Files': len(files),
            'Size (KB)': total_size // 1024
        })
    
    df_runs = pd.DataFrame(runs_info)
    print(df_runs.to_string(index=False))
    print("=" * 80)
    print(f"\nTotal data: {df_runs['Size (KB)'].sum():,} KB")
else:
    print("Skipping data exploration")
    print("   Set ENABLE_DATA_EXPLORATION = True to view data details")

### Inspect Latest Run Details

In [ ]:
if ENABLE_DATA_EXPLORATION:
    # Load latest run
    latest_run = run_dirs[0]
    print(f"Latest Run: {latest_run.name}\n")
    print("=" * 80)
    
    # Load all JSON files
    files_content = {}
    for json_file in latest_run.glob('*.json'):
        with open(json_file, 'r', encoding='utf-8') as f:
            files_content[json_file.stem] = json.load(f)
        print(f"OK Loaded: {json_file.name}")
    
    # Display summary
    print("\nData Summary:")
    print("=" * 80)
    
    if 'nodes' in files_content:
        nodes = files_content['nodes']
        print(f"Nodes (Intersections): {len(nodes)}")
        if nodes:
            print(f"   Sample: {nodes[0].get('name', 'N/A')}")
            print(f"   Location: ({nodes[0]['lat']:.6f}, {nodes[0]['lon']:.6f})")
    
    if 'edges' in files_content:
        edges = files_content['edges']
        print(f"\nEdges (Road Segments): {len(edges)}")
        if edges:
            sample = edges[0]
            print(f"   Sample: {sample.get('road_name', 'Unnamed road')}")
            print(f"   Route: {sample.get('start_node_id', 'N/A')} → {sample.get('end_node_id', 'N/A')}")
            print(f"   Distance: {sample.get('distance_m', 0):.1f} m")
    
    if 'traffic_edges' in files_content:
        traffic = files_content['traffic_edges']
        print(f"\nTraffic Data: {len(traffic)} records")
        if traffic:
            sample = traffic[0]
            print(f"   Speed: {sample.get('speed_kmh', 'N/A')} km/h")
            print(f"   Duration: {sample.get('duration_sec', 'N/A')} sec")
            print(f"   Distance: {sample.get('distance_km', 'N/A')} km")
            print(f"   Timestamp: {sample.get('timestamp', 'N/A')}")
    
    if 'weather_snapshot' in files_content:
        weather = files_content['weather_snapshot']
        print(f"\nWeather Data: {len(weather)} node records")
        if weather:
            sample = weather[0]
            print(f"   Temperature: {sample.get('temperature_c', 'N/A')}°C")
            print(f"   Wind Speed: {sample.get('wind_speed_kmh', 'N/A')} km/h")
            print(f"   Precipitation: {sample.get('precipitation_mm', 'N/A')} mm")
    
    if 'statistics' in files_content:
        stats = files_content['statistics']
        print(f"\nNetwork Statistics:")
        print(f"   Total Nodes: {stats.get('total_nodes', 'N/A')}")
        print(f"   Total Edges: {stats.get('total_edges', 'N/A')}")
        print(f"   Avg Degree: {stats.get('avg_degree', 'N/A'):.2f}")
    
    print("\n" + "=" * 80)
else:
    print("Skipping detailed exploration")

---
## Step 4️: Data Preprocessing

**Convert JSON → Parquet** for 10x faster loading + add derived features.

In [ ]:
if ENABLE_PREPROCESSING and not USE_PREPROCESSED:
    import sys
    sys.path.insert(0, '..')  # Add project root to path
    
    from pathlib import Path
    import pandas as pd
    import numpy as np
    import json
    
    print("Preprocessing data...\n")
    print("=" * 80)
    
    # Create preprocessor class (inline for notebook)
    class SimplePreprocessor:
        def __init__(self, data_dir=DATA_DIR, output_dir=PROCESSED_DIR):
            self.data_dir = Path(data_dir)
            self.output_dir = Path(output_dir)
            self.output_dir.mkdir(parents=True, exist_ok=True)
        
        def process_run(self, run_dir):
            """Process a single run directory"""
            run_name = run_dir.name
            if VERBOSE:
                print(f"Processing: {run_name}")
            
            # Load JSON files
            with open(run_dir / 'traffic_edges.json', 'r') as f:
                traffic_data = json.load(f)
            
            with open(run_dir / 'weather_snapshot.json', 'r') as f:
                weather_data = json.load(f)
            
            with open(run_dir / 'nodes.json', 'r') as f:
                nodes_data = json.load(f)
            
            # Convert to DataFrames
            df_traffic = pd.DataFrame(traffic_data)
            df_weather = pd.DataFrame(weather_data)
            df_nodes = pd.DataFrame(nodes_data)
            
            # Parse timestamp
            df_traffic['timestamp'] = pd.to_datetime(df_traffic['timestamp'])
            
            # Add time-based features
            df_traffic['hour'] = df_traffic['timestamp'].dt.hour
            df_traffic['minute'] = df_traffic['timestamp'].dt.minute
            df_traffic['day_of_week'] = df_traffic['timestamp'].dt.dayofweek
            df_traffic['day_name'] = df_traffic['timestamp'].dt.day_name()
            df_traffic['is_weekend'] = df_traffic['day_of_week'].isin([5, 6])
            
            # Add congestion levels based on speed
            def categorize_congestion(speed):
                if speed < 15:
                    return 'heavy'
                elif speed < 25:
                    return 'moderate'
                elif speed < 35:
                    return 'light'
                else:
                    return 'free_flow'
            
            df_traffic['congestion_level'] = df_traffic['speed_kmh'].apply(categorize_congestion)
            
            # Add speed categories
            df_traffic['speed_category'] = pd.cut(
                df_traffic['speed_kmh'],
                bins=[0, 10, 20, 30, 40, 100],
                labels=['very_slow', 'slow', 'moderate', 'fast', 'very_fast']
            )
            
            # Merge weather data (average across all nodes)
            weather_avg = df_weather.groupby('node_id').first().reset_index()
            avg_temp = weather_avg['temperature_c'].mean()
            avg_wind = weather_avg['wind_speed_kmh'].mean()
            avg_precip = weather_avg['precipitation_mm'].mean()
            
            df_traffic['temperature_c'] = avg_temp
            df_traffic['wind_speed_kmh'] = avg_wind
            df_traffic['precipitation_mm'] = avg_precip
            
            # Add run metadata
            df_traffic['run_name'] = run_name
            df_traffic['collection_time'] = df_traffic['timestamp'].iloc[0]
            
            # Save to Parquet
            output_file = self.output_dir / f"{run_name}.parquet"
            df_traffic.to_parquet(output_file, index=False)
            if VERBOSE:
                print(f"   OK Saved: {output_file.name} ({output_file.stat().st_size // 1024} KB)")
            
            return df_traffic
        
        def process_all(self, limit=None):
            """Process all runs"""
            run_dirs = sorted([d for d in self.data_dir.iterdir() if d.is_dir()], reverse=True)
            
            if limit:
                run_dirs = run_dirs[:limit]
            
            all_data = []
            for run_dir in run_dirs:
                try:
                    df = self.process_run(run_dir)
                    all_data.append(df)
                except Exception as e:
                    if VERBOSE:
                        print(f"   Error: {e}")
            
            # Combine all runs
            if all_data:
                df_combined = pd.concat(all_data, ignore_index=True)
                combined_file = self.output_dir / 'all_runs_combined.parquet'
                df_combined.to_parquet(combined_file, index=False)
                print(f"\nCombined dataset: {combined_file.name}")
                print(f"   Records: {len(df_combined):,}")
                print(f"   Size: {combined_file.stat().st_size // 1024} KB")
                return df_combined
            
            return None
    
    # Run preprocessing
    preprocessor = SimplePreprocessor()
    df_all = preprocessor.process_all()
    
    print("\n" + "=" * 80)
    print("OKPreprocessing completed!")
    print(f"\nTotal records: {len(df_all):,}")
    print(f"Date range: {df_all['timestamp'].min()} to {df_all['timestamp'].max()}")
    print(f"Avg speed: {df_all['speed_kmh'].mean():.2f} km/h")

elif USE_PREPROCESSED:
    # Load from preprocessed Parquet files
    print("Loading preprocessed data (fast)...\n")
    combined_file = Path(PROCESSED_DIR) / 'all_runs_combined.parquet'
    if combined_file.exists():
        df_all = pd.read_parquet(combined_file)
        print(f"OKLoaded {len(df_all):,} records from {combined_file.name}")
        print(f"Date range: {df_all['timestamp'].min()} to {df_all['timestamp'].max()}")
        print(f"Avg speed: {df_all['speed_kmh'].mean():.2f} km/h")
    else:
        print(f"FAIL Preprocessed file not found: {combined_file}")
        print("   Set ENABLE_PREPROCESSING = True to create it")
else:
    print("Skipping preprocessing")
    print("   Set ENABLE_PREPROCESSING = True to process data")

### Preview Preprocessed Data

In [ ]:
print("Dataset Info:\n")
print(df_all.info())

print("\n" + "=" * 80)
print("\nFirst 5 rows:\n")
print(df_all.head())

print("\n" + "=" * 80)
print("\nStatistical Summary:\n")
print(df_all[['speed_kmh', 'duration_sec', 'distance_km', 'temperature_c', 'wind_speed_kmh']].describe())

---
## Step 5️: Comprehensive Exploratory Data Analysis

**Deep dive into patterns** - Geographic maps, correlations, time series.

In [ ]:
if ENABLE_COMPREHENSIVE_EDA:
    print("Running Comprehensive EDA...\n")
    print("=" * 70)
    
    # Import visualization libraries
    if SHOW_PLOTLY_CHARTS:
        import plotly.express as px
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
    
    if SHOW_INTERACTIVE_MAPS:
        import folium
        from folium.plugins import HeatMap
    
    print("OK Visualization libraries loaded")
else:
    print("Skipping comprehensive EDA")
    print("Set ENABLE_COMPREHENSIVE_EDA = True for detailed analysis")

### 5a. Traffic Speed Distribution

In [ ]:
if ENABLE_COMPREHENSIVE_EDA and SHOW_PLOTLY_CHARTS:
    # Traffic Speed Analysis
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Speed Distribution', 'Speed Box Plot', 
                        'Duration Distribution', 'Speed vs Distance'),
        specs=[[{'type': 'histogram'}, {'type': 'box'}],
               [{'type': 'histogram'}, {'type': 'scatter'}]]
    )
    
    # Speed Histogram
    fig.add_trace(
        go.Histogram(x=df_all['speed_kmh'], nbinsx=30, name='Speed',
                    marker_color='steelblue', showlegend=False),
        row=1, col=1
    )
    
    # Speed Box Plot
    fig.add_trace(
        go.Box(y=df_all['speed_kmh'], name='Speed',
               marker_color='steelblue', showlegend=False),
        row=1, col=2
    )
    
    # Duration Histogram
    fig.add_trace(
        go.Histogram(x=df_all['duration_sec'], nbinsx=30, name='Duration',
                    marker_color='coral', showlegend=False),
        row=2, col=1
    )
    
    # Speed vs Distance Scatter
    fig.add_trace(
        go.Scatter(x=df_all['distance_km'], y=df_all['speed_kmh'],
                  mode='markers', name='Speed vs Distance',
                  marker=dict(color='green', size=6, opacity=0.4),
                  showlegend=False),
        row=2, col=2
    )
    
    fig.update_xaxes(title_text="Speed (km/h)", row=1, col=1)
    fig.update_xaxes(title_text="Speed (km/h)", row=1, col=2)
    fig.update_xaxes(title_text="Duration (seconds)", row=2, col=1)
    fig.update_xaxes(title_text="Distance (km)", row=2, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Speed (km/h)", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_yaxes(title_text="Speed (km/h)", row=2, col=2)
    
    fig.update_layout(height=700, title_text="Traffic Speed & Duration Analysis", showlegend=False)
    fig.show()
    
    # Print congestion statistics
    print("\nCongestion Level Distribution:")
    print(df_all['congestion_level'].value_counts().sort_index())
    print(f"\nAverage Speed: {df_all['speed_kmh'].mean():.2f} km/h")
    print(f"Average Duration: {df_all['duration_sec'].mean():.2f} sec")

### 5b. Hourly Traffic Patterns

In [ ]:
if ENABLE_COMPREHENSIVE_EDA:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Hourly patterns
    hourly_speed = df_all.groupby('hour')['speed_kmh'].agg(['mean', 'std', 'count']).reset_index()
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    fig.suptitle('Hourly Traffic Patterns', fontsize=16, fontweight='bold')
    
    # Average speed by hour
    axes[0].plot(hourly_speed['hour'], hourly_speed['mean'], marker='o', linewidth=2, markersize=8, color='steelblue')
    axes[0].fill_between(
        hourly_speed['hour'],
        hourly_speed['mean'] - hourly_speed['std'],
        hourly_speed['mean'] + hourly_speed['std'],
        alpha=0.3,
        color='steelblue'
    )
    axes[0].set_xlabel('Hour of Day', fontsize=12)
    axes[0].set_ylabel('Average Speed (km/h)', fontsize=12)
    axes[0].set_title('Average Speed by Hour (with std deviation)')
    axes[0].set_xticks(range(0, 24, 2))
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(y=df_all['speed_kmh'].mean(), color='red', linestyle='--', label='Overall Average')
    axes[0].legend()
    
    # Traffic volume by hour
    axes[1].bar(hourly_speed['hour'], hourly_speed['count'], color='coral', alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Hour of Day', fontsize=12)
    axes[1].set_ylabel('Number of Records', fontsize=12)
    axes[1].set_title('Traffic Data Volume by Hour')
    axes[1].set_xticks(range(0, 24, 2))
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Peak hours identification
    peak_hours = hourly_speed.nsmallest(3, 'mean')[['hour', 'mean']]
    print("\nSlowest Hours (Peak Congestion):")
    for _, row in peak_hours.iterrows():
        print(f"   • Hour {int(row['hour']):02d}:00 - Avg Speed: {row['mean']:.2f} km/h")

---
## Step 6️: Feature Engineering for ML

**Create advanced features** for machine learning models.

In [ ]:
if ENABLE_FEATURE_ENGINEERING:
    from sklearn.preprocessing import LabelEncoder
    
    print("Engineering features...\n")
    print("=" * 70)
    
    # Create edge_id from node pairs (if not exists)
    if 'edge_id' not in df_all.columns:
        df_all['edge_id'] = df_all['node_a_id'] + '_to_' + df_all['node_b_id']
        print("✓ Created edge_id from node_a_id and node_b_id")
    
    # Sort by timestamp for lag features
    df_all = df_all.sort_values(['edge_id', 'timestamp']).reset_index(drop=True)
    
    # 1. Lag features (previous speeds)
    print("\n1️ Creating lag features...")
    for lag in [1, 2, 3]:
        df_all[f'speed_lag_{lag}'] = df_all.groupby('edge_id')['speed_kmh'].shift(lag)
    print(f"   OK Added: speed_lag_1, speed_lag_2, speed_lag_3")
    
    # 2. Rolling statistics
    print("\n2️ Creating rolling statistics...")
    df_all['speed_rolling_mean_3'] = df_all.groupby('edge_id')['speed_kmh'].transform(
        lambda x: x.rolling(window=3, min_periods=1).mean()
    )
    df_all['speed_rolling_std_3'] = df_all.groupby('edge_id')['speed_kmh'].transform(
        lambda x: x.rolling(window=3, min_periods=1).std()
    )
    print(f"   OK Added: speed_rolling_mean_3, speed_rolling_std_3")
    
    # 3. Cyclical time features
    print("\n3️ Creating cyclical time features...")
    df_all['hour_sin'] = np.sin(2 * np.pi * df_all['hour'] / 24)
    df_all['hour_cos'] = np.cos(2 * np.pi * df_all['hour'] / 24)
    df_all['day_of_week_sin'] = np.sin(2 * np.pi * df_all['day_of_week'] / 7)
    df_all['day_of_week_cos'] = np.cos(2 * np.pi * df_all['day_of_week'] / 7)
    print(f"   OK Added: hour_sin, hour_cos, day_of_week_sin, day_of_week_cos")
    
    # 4. Rush hour indicators
    print("\n4️ Creating rush hour indicators...")
    df_all['is_morning_rush'] = df_all['hour'].isin([7, 8, 9]).astype(int)
    df_all['is_evening_rush'] = df_all['hour'].isin([17, 18, 19]).astype(int)
    df_all['is_rush_hour'] = (df_all['is_morning_rush'] | df_all['is_evening_rush']).astype(int)
    print(f"   OK Added: is_morning_rush, is_evening_rush, is_rush_hour")
    
    # 5. Encode categorical features
    print("\n5️ Encoding categorical features...")
    le_congestion = LabelEncoder()
    df_all['congestion_level_encoded'] = le_congestion.fit_transform(df_all['congestion_level'])
    print(f"   OK Added: congestion_level_encoded")
    
    # Drop rows with NaN (from lag features)
    df_features = df_all.dropna().copy()
    
    print("\n" + "=" * 70)
    print(f"OKFeature engineering completed!")
    print(f"\nDataset shape: {df_features.shape}")
    print(f"Total features: {df_features.shape[1]}")
    print(f"Dropped {len(df_all) - len(df_features)} rows with missing lag values")
else:
    print("Skipping feature engineering")
    print("WARNING: Required for model training!")
    df_features = df_all.copy()

### Feature List

In [ ]:
print("All Features:\n")
print("=" * 80)

feature_groups = {
    'Traffic Features': ['speed_kmh', 'duration_sec', 'distance_km', 'congestion_level', 'speed_category'],
    'Time Features': ['hour', 'minute', 'day_of_week', 'day_name', 'is_weekend', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos'],
    'Weather Features': ['temperature_c', 'wind_speed_kmh', 'precipitation_mm'],
    'Lag Features': ['speed_lag_1', 'speed_lag_2', 'speed_lag_3'],
    'Rolling Features': ['speed_rolling_mean_3', 'speed_rolling_std_3'],
    'Rush Hour Features': ['is_morning_rush', 'is_evening_rush', 'is_rush_hour'],
    'Encoded Features': ['congestion_level_encoded']
}

for group_name, features in feature_groups.items():
    print(f"\n{group_name}:")
    for feat in features:
        if feat in df_features.columns:
            print(f"   OK {feat}")
        else:
            print(f"   NOT {feat} (missing)")

print("\n" + "=" * 80)

---
## Step 7️: Train & Compare Models

**Train multiple models** and compare their performance.

In [ ]:
if ENABLE_MODEL_TRAINING:
    from sklearn.model_selection import train_test_split
    
    print("Splitting data...\n")
    print("=" * 70)
    
    # Define features for modeling
    feature_columns = [
        # Time features
        'hour', 'day_of_week', 'is_weekend', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
        # Traffic features
        'distance_km', 'duration_sec',
        # Weather features
        'temperature_c', 'wind_speed_kmh', 'precipitation_mm',
        # Lag features
        'speed_lag_1', 'speed_lag_2', 'speed_lag_3',
        # Rolling features
        'speed_rolling_mean_3', 'speed_rolling_std_3',
        # Rush hour
        'is_morning_rush', 'is_evening_rush', 'is_rush_hour'
    ]
    
    target_column = 'speed_kmh'
    
    # Prepare X and y
    X = df_features[feature_columns].copy()
    y = df_features[target_column].copy()
    
    # Time-based split (last 20% as test)
    split_idx = int(len(df_features) * 0.8)
    X_train = X.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_train = y.iloc[:split_idx]
    y_test = y.iloc[split_idx:]
    
    print(f"Dataset split (time-based):") 
    print(f"   Training set: {len(X_train):,} samples ({len(X_train)/len(X)*100:.1f}%)")
    print(f"   Test set: {len(X_test):,} samples ({len(X_test)/len(X)*100:.1f}%)")
    print(f"\nFeatures: {len(feature_columns)}")
    print(f"Target: {target_column}")
else:
    print("Skipping train-test split")
    print("   Set ENABLE_MODEL_TRAINING = True to train models")

### Model Comparison Results

In [ ]:
if ENABLE_MODEL_TRAINING:
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.linear_model import LinearRegression, Ridge
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import time
    
    print("Training models...\n")
    print("=" * 70)
    
    # Define models
    if QUICK_MODE:
        models = {
            'Linear Regression': LinearRegression(),
            'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1),
        }
        print("Quick Mode: Training 2 fast models\n")
    else:
        models = {
            'Linear Regression': LinearRegression(),
            'Ridge Regression': Ridge(alpha=1.0),
            'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
        }
        print("Full Mode: Training 4 models\n")
    
    results = []
    
    for model_name, model in models.items():
        print(f"Training: {model_name}")
        
        # Train
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Evaluate
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Model': model_name,
            'MAE': mae,
            'RMSE': rmse,
            'R²': r2,
            'Train Time (s)': train_time
        })
        
        print(f"   OK MAE: {mae:.3f} km/h")
        print(f"   OK RMSE: {rmse:.3f} km/h")
        print(f"   OK R²: {r2:.3f}")
        print(f"   Time: {train_time:.2f}s\n")
    
    print("=" * 70)
    print("OKAll models trained!")
else:
    print("Skipping model training")
    print("Set ENABLE_MODEL_TRAINING = True to train models")

### Model Comparison

In [ ]:
if ENABLE_MODEL_TRAINING:
    # Create comparison table
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values('RMSE')
    
    print("\nModel Performance Comparison:\n")
    print("=" * 70)
    print(df_results.to_string(index=False))
    print("=" * 70)
    
    best_model_name = df_results.iloc[0]['Model']
    best_rmse = df_results.iloc[0]['RMSE']
    best_r2 = df_results.iloc[0]['R²']
    
    print(f"\nBest Model: {best_model_name}")
    print(f"   RMSE: {best_rmse:.3f} km/h")
    print(f"   R²: {best_r2:.3f}")

### Performance Visualization

In [ ]:
if ENABLE_MODEL_TRAINING:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('Model Performance Metrics', fontsize=16, fontweight='bold')
    
    # MAE comparison
    axes[0].barh(df_results['Model'], df_results['MAE'], color='skyblue')
    axes[0].set_xlabel('MAE (km/h) - Lower is Better')
    axes[0].set_title('Mean Absolute Error')
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # RMSE comparison
    axes[1].barh(df_results['Model'], df_results['RMSE'], color='lightcoral')
    axes[1].set_xlabel('RMSE (km/h) - Lower is Better')
    axes[1].set_title('Root Mean Squared Error')
    axes[1].grid(True, alpha=0.3, axis='x')
    
    # R² comparison
    axes[2].barh(df_results['Model'], df_results['R²'], color='lightgreen')
    axes[2].set_xlabel('R² Score - Higher is Better')
    axes[2].set_title('R² Score')
    axes[2].set_xlim(0, 1)
    axes[2].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()

---
## Step 8️: Save Models & Results

**Export trained models** for deployment and future use.

In [ ]:
if ENABLE_SAVE_MODELS and ENABLE_MODEL_TRAINING:
    import joblib
    from pathlib import Path
    
    # Create models directory
    models_dir = Path(MODELS_DIR)
    models_dir.mkdir(parents=True, exist_ok=True)
    
    print("Saving models...\n")
    print("=" * 70)
    
    # Save all models
    for model_name, model in models.items():
        filename = model_name.lower().replace(' ', '_') + '.pkl'
        filepath = models_dir / filename
        joblib.dump(model, filepath)
        print(f"OK Saved: {filename} ({filepath.stat().st_size // 1024} KB)")
    
    # Save feature columns
    feature_info = {
        'feature_columns': feature_columns,
        'target_column': target_column,
        'num_features': len(feature_columns)
    }
    joblib.dump(feature_info, models_dir / 'feature_info.pkl')
    print(f"\nOK Saved feature info")
    
    # Save results
    df_results.to_csv(models_dir / 'model_comparison.csv', index=False)
    print(f"OK Saved model comparison")
    
    print("\n" + "=" * 70)
    print(f"OKAll artifacts saved to: {models_dir}")
elif not ENABLE_MODEL_TRAINING:
    print("No models to save (training was skipped)")
else:
    print("Skipping model saving")
    print("   Set ENABLE_SAVE_MODELS = True to save models")

---
## Pipeline Complete!

**Summary of execution** and next steps.

In [ ]:
print("╔" + "═" * 68 + "╗")
print("║" + " " * 20 + "PIPELINE EXECUTION COMPLETE!" + " " * 17 + "║")
print("╚" + "═" * 68 + "╝")
print()

# Summary
print("EXECUTION SUMMARY")
print("=" * 70)

steps_summary = [
    ("Configuration", True, "OKLoaded"),
    ("Download Data", not USE_EXISTING_DATA, "OKDownloaded" if not USE_EXISTING_DATA else "Skipped (used existing)"),
    ("Data Exploration", ENABLE_DATA_EXPLORATION, "OKExplored" if ENABLE_DATA_EXPLORATION else "Skipped"),
    ("Preprocessing", ENABLE_PREPROCESSING, "OKProcessed" if ENABLE_PREPROCESSING else "Skipped"),
    ("Comprehensive EDA", ENABLE_COMPREHENSIVE_EDA, "OKAnalyzed" if ENABLE_COMPREHENSIVE_EDA else "Skipped"),
    ("Feature Engineering", ENABLE_FEATURE_ENGINEERING, f"OKCreated {len(feature_columns)} features" if ENABLE_FEATURE_ENGINEERING else "Skipped"),
    ("Model Training", ENABLE_MODEL_TRAINING, f"OKTrained {len(models)} models" if ENABLE_MODEL_TRAINING else "Skipped"),
    ("Save Models", ENABLE_SAVE_MODELS and ENABLE_MODEL_TRAINING, "OKSaved" if ENABLE_SAVE_MODELS and ENABLE_MODEL_TRAINING else "Skipped"),
]

for step_name, executed, status in steps_summary:
    print(f"• {step_name:.<25} {status}")

if ENABLE_MODEL_TRAINING:
    print("\nBEST MODEL RESULTS")
    print("=" * 70)
    print(f"• Model: {best_model_name}")
    print(f"• RMSE: {best_rmse:.3f} km/h")
    print(f"• MAE: {df_results.iloc[0]['MAE']:.3f} km/h")
    print(f"• R² Score: {best_r2:.3f}")
    print(f"• Training Time: {df_results.iloc[0]['Train Time (s)']:.2f}s")

print("\nNEXT STEPS")
print("=" * 70)
print("1. Hyperparameter Tuning - Optimize best model with GridSearch")
print("2. Advanced Models - Try LSTM, GNN for temporal/spatial patterns")
print("3. Ensemble Methods - Combine multiple models")
print("4. Deploy API - Create endpoint for real-time predictions")
print("5. Dashboard - Build real-time monitoring dashboard")

print("\nTO RE-RUN THIS PIPELINE")
print("=" * 70)
print("• Just 'Run All Cells' again!")
print("• Or adjust configuration in Step 1 and re-run")
print("• Set USE_EXISTING_DATA = False to download fresh data")
print("• Enable/disable steps as needed")

print("\n" + "=" * 70)
print("Thank you for using the Traffic Forecasting Pipeline!")
print("=" * 70)